In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Hello World: Dein erster Quantenschaltkreis

Baue einen [Bell-Zustand](https://en.wikipedia.org/wiki/Bell_state) (zwei verschränkte Qubits) und führe ihn auf drei Arten aus:

1. **Ideale Simulation** — perfekte Ergebnisse, kein Konto erforderlich
2. **Simulation mit Rauschen** — simuliert ein echtes Gerät, kein Konto erforderlich
3. **Echte Quantenhardware** — erfordert ein [IBM Quantum-Konto](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account)

## Schaltkreis erstellen

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

qc.draw(output="mpl")

## Option 1: Ideale Simulation (kein Konto erforderlich)
Verwendet `StatevectorSampler` — ein lokaler Simulator mit perfekten, rauschfreien Ergebnissen.

In [ ]:
from qiskit.primitives import StatevectorSampler

result = StatevectorSampler().run([qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram(counts)

## Option 2: Simulation mit Rauschen (kein Konto erforderlich)
Verwendet `FakeManilaV2` — ein lokaler Simulator, der ein echtes IBM-Quantengerät einschließlich seiner Rauscheigenschaften nachahmt. Der Schaltkreis muss zunächst transpiliert (angepasst) werden, um dem Gate-Satz und der Qubit-Konnektivität des Geräts zu entsprechen.

In [ ]:
from qiskit_ibm_runtime import SamplerV2
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend = FakeManilaV2()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## Option 3: Echte Quantenhardware
Erfordert ein IBM Quantum-Konto. Siehe [Setting up your IBM Quantum account](https://janlahmann.github.io/Qiskit-documentation/#setting-up-your-ibm-quantum-account) für Details.

**Überspringe diese Zelle**, wenn du deine Zugangsdaten in den [Einstellungen](/jupyter-settings#ibm-quantum) gespeichert oder den Simulatormodus aktiviert hast — sie werden beim Klicken auf „Ausführen" automatisch eingefügt. Das Ausführen dieser Zelle mit Platzhalterwerten überschreibt deine Konfiguration.

Auf anderen Plattformen führe dies zuerst aus, um die Zugangsdaten für die Sitzung zu speichern:

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    token="<your-api-key>",
    instance="<your-crn>",
    overwrite=True
)

**Hinweis:** Jobs auf echter Hardware können je nach Wartezeiten in der Warteschlange einige Zeit benötigen. Falls die Zelle noch läuft, kannst du deinen Job-Status überprüfen und die Ergebnisse unter [quantum.cloud.ibm.com/workloads](https://quantum.cloud.ibm.com/workloads?user=me) einsehen.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)
print(f"Running on {backend.name}")

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

result = SamplerV2(mode=backend).run([isa_qc], shots=1024).result()
counts = result[0].data.meas.get_counts()
counts

In [ ]:
plot_histogram(counts)

## Was kommt als Nächstes?
- **[Tutorials](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=docs/tutorials)** — Schritt-für-Schritt-Anleitungen zu Algorithmen, Fehlerminderung, Transpilation und mehr
- **[Courses](https://mybinder.org/v2/gh/JanLahmann/Qiskit-documentation/main?filepath=learning/courses)** — Strukturierte Lernpfade von Quantengrundlagen bis zu Utility-Scale-Computing
- **[Local testing mode](https://janlahmann.github.io/Qiskit-documentation/#no-token-use-local-testing-mode)** — Die meisten Notebooks ohne IBM Quantum-Konto ausführen